In [13]:
#!pip install PyPDF2
#!pip install tinydb
#!pip install langchain
#!pip install langchain-community
#!pip install pandas


In [1]:
# %% [code]
import os
#from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import PyPDF2
from tinydb import TinyDB, Query
from langchain.chat_models import AzureChatOpenAI

# Configuración del LLM (ajusta el parámetro 'temperature' o agrega tus credenciales si es necesario)


llm = AzureChatOpenAI(
    openai_api_base="https://sg-openai.openai.azure.com/",
    openai_api_key="647e7eaa5678417f8667203df58216ee",  # Reemplaza con tu clave de API
    openai_api_version="2023-03-15-preview",  # Ajusta según la versión que estés usando
    deployment_name="gpt-4o",
    temperature=0.2
)

# Inicialización de TinyDB (la base se almacenará en el archivo history.json)
db = TinyDB("ResCTM.json")
records_table = db.table("records")


C:\Users\tschoppj\AppData\Local\Temp\ipykernel_3400\930510609.py:13: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
c:\ProyectoPY\AgentesAI\venv\Lib\site-packages\langchain_community\chat_models\azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://sg-openai.openai.azure.com/ to https://sg-openai.openai.azure.com/openai.
  warnings.warn(
c:\ProyectoPY\AgentesAI\venv\Lib\site-packages\langchain_community\chat_models\azure_openai.py:181: UserWarning: As of openai>=1.0.0, if `deployment_name` (or alias `azure_

# LLM local Ollama; Llama3.2 de 3b

In [43]:
# %% [code]
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import PyPDF2
from tinydb import TinyDB, Query
from langchain_ollama.llms import OllamaLLM

# Configuración del LLM para usar Ollama en la red interna
llm = OllamaLLM(
    model="llama3.2:latest",
    #base_url="http://localhost/11434/api ",#10.209.0.140:8000/api/proxy",  # Ajusta la IP y el puerto según tu configuración
    temperature=0.2
)

# Inicialización de TinyDB (la base se almacenará en el archivo ResCTM.json)
db = TinyDB("ResCTM.json")
records_table = db.table("records")

In [44]:
response = llm.predict("Hola, ¿cómo estás?")

print(response)

¡Hola! Estoy bien, gracias. Soy un modelo de lenguaje artificial, así que no tengo emociones o sentimientos como los humanos, pero estoy aquí para ayudarte con cualquier pregunta o tema que desees discutir. ¿En qué puedo ayudarte hoy?


# Funciones de procesamiento

In [45]:
# %% [code]
def pdf_to_text(pdf_path):
    """
    Extrae el texto de un archivo PDF utilizando PyPDF2.
    """
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text


# 1. Corrección de la transcripción usando LangChain

In [46]:
# %% [code]
correction_prompt = PromptTemplate(
    input_variables=["text"],
    template="Corrige la transcripción del siguiente texto:\n\n{text}\n\nTexto corregido:"
)

def correct_transcription(text):
    """
    Utiliza un LLM para corregir la transcripción extraída del PDF.
    """
    chain = LLMChain(llm=llm, prompt=correction_prompt)
    corrected = chain.run(text=text)
    return corrected


# 2. Extracción de datos y generación del resumen

In [47]:
# Prompt adaptado con instrucciones específicas
extraction_prompt = PromptTemplate(
    input_variables=["text", "template_info"],
    template=(
        "Utilizando el siguiente template:\n"
        "{template_info}\n\n"
        "Extrae únicamente los datos solicitados y genera un resumen ejecutivo breve (máximo 30 palabras) "
        "del tema principal del texto, sin incluir detalles menores ni secundarios.\n\n"
        "Texto:\n{text}\n\n"
        "Datos extraídos y resumen ejecutivo breve:"
    )
)

def extract_data_summary(text, extraction_template):
    """
    Extrae datos específicos y genera un resumen ejecutivo breve.
    """
    chain = LLMChain(llm=llm, prompt=extraction_prompt)
    result = chain.run(text=text, template_info=extraction_template)
    return result


# 3. Clasificación de la resolución

In [48]:
CATEGORIAS_CTMSG = {
    "Reglamentación y Normativas": [
        "REGLAMENTO",
        "MANUAL DE NORMA",
        "MANUAL Normas Compras y Contrataciones",
        "NORMA ISO",
        "LICENCIA ESPECIAL",
        "ESTATUTO DEL PERSONAL",
        "SUMARIO ADMINISTRATIVO"
    ],
    "Administración Financiera y Contable": [
        "REMUNERACIONES",
        "REGISTRACION CONTABLE",
        "BALANCES",
        "BANCOS",
        "PRESUPUESTOS",
        "TARIFAS ENERGÍA"
    ],
    "Contrataciones y Convenios": [
        "CONVENIO",
        "CONTRATACIONES Y COMPRAS",
        "COMPRAS AUTOMOTORES",
        "CONCURSOS PRECIOS",
        "PODERES ABOGADOS"
    ],
    "Infraestructura y Mantenimiento": [
        "EDIFICIOS",
        "VIVIENDAS",
        "EQUIPAMIENTO REPRESA",
        "MANTENIMIENTO",
        "TRANSFORMADORES",
        "SEGURIDAD PRESA",
        "PUERTO SECO"
    ],
    "Servicios Médicos y Sociales": [
        "PRESTAMOS PERSONAL",
        "SERVICIOS MÉDICOS",
        "ATENCIÓN MÉDICA",
        "SEGUROS VIDA"
    ],
    "Comunicaciones y Relaciones Institucionales": [
        "RELACIONES PÚBLICAS",
        "COMUNICACIÓN INSTITUCIONAL",
        "PUBLICIDAD",
        "LOGOTIPO",
        "FIRMA DIGITAL",
        "PAGINA WEB"
    ],
    "Auditorías y Control Interno": [
        "AUDITORIA INTERNA",
        "AUDITORÍA EXTERNA",
        "CONTROL GESTIÓN"
    ],
    "Recursos Humanos": [
        "CONTROL ASISTENCIA",
        "EPIDEMIOLOGÍA",
        "CURSOS Y SEMINARIOS",
        "DESVINCULACIONES"
    ],
    "Ambiental y Social": [
        "RIBEREÑOS",
        "MERCOSUR",
        "MERCADOS ELÉCTRICOS",
        "CULTURALES",
        "SAOTTA"
    ],
    "Gestión y Operaciones": [
        "DIRECCIÓN EJECUTIVA",
        "GERENCIA",
        "DELEGACIONES",
        "TAISG",
        "HORARIOS OFICINAS",
        "GRUPOS TRABAJO",
        "CENTRO DOCUMENTACIÓN",
        "RENOVACIÓN SALTO GRANDE"
    ]
}


In [49]:
# Prompt adaptado
classification_prompt = PromptTemplate(
    input_variables=["text", "data_summary", "categorias"],
    template=(
        "Basándote en el siguiente texto y en los datos extraídos y resumen:\n\n"
        "Texto: {text}\n\n"
        "Datos y resumen: {data_summary}\n\n"
        "Considerando estas categorías disponibles: {categorias}\n\n"
        "Clasifica la resolución estrictamente dentro de una de estas categorías principales."
    )
)

# Función ajustada
def classify_resolution(text, data_summary):
    categorias_str = ', '.join(CATEGORIAS_CTMSG.keys())
    chain = LLMChain(llm=llm, prompt=classification_prompt)
    classification = chain.run(text=text, data_summary=data_summary, categorias=categorias_str)
    return classification


# Función principal para procesar el PDF y guardar la información en TinyDB

In [50]:
import re

def parse_data_summary(data_summary):
    """
    Divide el resumen extraído en campos separados.
    """
    fields = ["Número de resolución", "Fecha", "Ente emisor", "Resumen ejecutivo breve"]
    parsed_data = {}

    for field in fields:
        pattern = rf"{field}:\s*(.+)"
        match = re.search(pattern, data_summary)
        if match:
            parsed_data[field] = match.group(1).strip()
        else:
            parsed_data[field] = None

    return parsed_data

In [51]:
# %% [code]
def process_pdf(pdf_path, extraction_template, db_table):
    """
    Procesa un archivo PDF completo: extrae el texto, lo corrige,
    extrae datos y resumen, clasifica la resolución y guarda todo en TinyDB.
    """
    file_name = os.path.basename(pdf_path)
    
    # Verificar si el documento ya se encuentra en la base de datos
    Record = Query()
    if db_table.search(Record.pdf_file == file_name):
        print(f"El documento {file_name} ya se encuentra registrado en TinyDB.")
        return

    # Extracción de texto
    raw_text = pdf_to_text(pdf_path)
    print(f"Texto extraído del archivo {file_name}.")
    
    # Corrección de la transcripción
    corrected_text = correct_transcription(raw_text)
    print("Transcripción corregida.")
    
    # Extracción de datos y resumen
    extracted_result = extract_data_summary(corrected_text, extraction_template)
    print("Datos y resumen extraídos.")
    
    # Clasificación de la resolución
    classification = classify_resolution(corrected_text, extracted_result)
    print("Clasificación realizada.")
    
    # Crear el registro con la información resultante
    parsed_summary = parse_data_summary(extracted_result)

    record = {
        "pdf_file": file_name,
        "text": corrected_text,
        "numero_resolucion": parsed_summary["Número de resolución"],
        "fecha": parsed_summary["Fecha"],
        "ente_emisor": parsed_summary["Ente emisor"],
        "resumen_ejecutivo": parsed_summary["Resumen ejecutivo breve"],
        "classification": classification
    }
    
    # Guardar el registro en TinyDB
    db_table.insert(record)
    print(f"El documento {file_name} ha sido registrado en TinyDB.")

    return record


# Ejecución del procesamiento

In [52]:
# %% [code]
# Template explícito para extracción
extraction_template = (
    "- Número de resolución:\n"
    "- Fecha:\n"
    "- Ente emisor:\n"
    "- Resumen ejecutivo breve (máximo 30 palabras, tema principal solamente):"
)

# Carpeta donde se encuentran los archivos PDF
pdf_folder = "pdfs"  # Asegúrate de tener esta carpeta y colocar allí tus PDFs

# Procesar cada PDF nuevo en la carpeta
for file_name in os.listdir(pdf_folder):
    if file_name.lower().endswith(".pdf"):
        pdf_path = os.path.join(pdf_folder, file_name)
        process_pdf(pdf_path, extraction_template, records_table)


El documento res00122.pdf ya se encuentra registrado en TinyDB.
El documento res00222.pdf ya se encuentra registrado en TinyDB.


# Función para procesar un nuevo PDF individualmente

In [58]:
# %% [code]
def procesar_nuevo_pdf(pdf_path, extraction_template, db_table):
    """
    Procesa un nuevo PDF individualmente y lo añade a la base de datos TinyDB.
    """
        # Crear el registro con la información resultante
    record = {
        "pdf_file": "New Resolucion",
        "text": "corrected_text",
        "data_summary": "extracted_result",
        "classification": "classification"
    }

    file_name = os.path.basename(pdf_path)
    Record = Query()
    if db_table.search(Record.pdf_file == file_name):
        print(f"El documento {file_name} ya se encuentra registrado.")
        return
    else:
        record = process_pdf(pdf_path, extraction_template, db_table)
        print(record)
        

#Ejemplo de llamada para procesar un nuevo PDF:
nuevo_pdf = "pdfs/new/res00722.pdf"
procesar_nuevo_pdf(nuevo_pdf, extraction_template, records_table)



Texto extraído del archivo res00722.pdf.
Transcripción corregida.
Datos y resumen extraídos.
Clasificación realizada.
El documento res00722.pdf ha sido registrado en TinyDB.
{'pdf_file': 'res00722.pdf', 'text': 'Aquí te presento el texto corregido:\n\nCOMISIÓN TECNICA MIXTA DE SALTO GRANDE\nVideoconferencia, 26 de enero de 2022\nRESOLUCIÓN CTM Nº 007/22 (Acta Nº 1127 - as. 8.7)\n\nVISTO: el EE 2021-ALET-0305 y el Memorándum AL N° 395/2021, por el cual se informa y solicita la conformidad de C.T.M.S.G. para el otorgamiento de la escritura definitiva del padrón urbano 28175, Unidad 004, Block C, ubicado en el complejo de viviendas construido por Salto Grande en el barrio San Martín de la Ciudad de Salto y disponer el pago del monto retenido a los presuntos herederos del Señor Hugo Torres García; y\n\nCONSIDERANDO: que, con fecha 22 de marzo de 1995 se celebró entre la C.T.M.S.G. y el ex funcionario Señor Hugo Torres García, un compromiso de compraventa respecto del citado padrón urbano 2

# Inspeccion de DB

In [59]:
import pandas as pd

db = TinyDB("ResCTM.json")
records_table = db.table("records")

registro = records_table.get(doc_id=7)

print(registro)

if registro:
    for campo, valor in registro.items():
        print(f"{campo}:")
        if isinstance(valor, str) and len(valor) > 300:
            print(valor[:300] + "...")
        else:
            print(valor)
        print("-" * 100)
else:
    print("Registro no encontrado.")



{'pdf_file': 'res00722.pdf', 'text': 'Aquí te presento el texto corregido:\n\nCOMISIÓN TECNICA MIXTA DE SALTO GRANDE\nVideoconferencia, 26 de enero de 2022\nRESOLUCIÓN CTM Nº 007/22 (Acta Nº 1127 - as. 8.7)\n\nVISTO: el EE 2021-ALET-0305 y el Memorándum AL N° 395/2021, por el cual se informa y solicita la conformidad de C.T.M.S.G. para el otorgamiento de la escritura definitiva del padrón urbano 28175, Unidad 004, Block C, ubicado en el complejo de viviendas construido por Salto Grande en el barrio San Martín de la Ciudad de Salto y disponer el pago del monto retenido a los presuntos herederos del Señor Hugo Torres García; y\n\nCONSIDERANDO: que, con fecha 22 de marzo de 1995 se celebró entre la C.T.M.S.G. y el ex funcionario Señor Hugo Torres García, un compromiso de compraventa respecto del citado padrón urbano 28175;\n\nque, el 31 de agosto de 2001, el Sr. Torres canceló la totalidad de precio del compromiso de compraventa, abonando todas las cuotas, tal como se desprende del inform

In [55]:
#db = TinyDB('historial.json')
#db.truncate()  # Esto vacía completamente la base de datos
#db.close()